# Download and prepare dataset directory

In [1]:
import os

## Mount your drive to store images and model

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")
!mkdir /content/gdrive/MyDrive/controlnet_dataset

Mounted at /content/gdrive


In [3]:
#!mkdir /content/gdrive/MyDrive/controlnet_dataset

mkdir: cannot create directory ‘/content/gdrive/MyDrive/controlnet_dataset’: File exists


### Dataset download
- Here we are using ImageNetDog dataset which contains approx 20k images of dogs
- It can take some time while unzipping all the images to the the directory

In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar -P /content/gdrive/MyDrive/controlnet_dataset
!tar -C /content/gdrive/MyDrive/controlnet_dataset -xvf /content/gdrive/MyDrive/controlnet_dataset/images.tar

--2023-05-25 18:22:21--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘/content/gdrive/MyDrive/controlnet_dataset/images.tar’

images.tar          100%[===================>] 756.82M  23.6MB/s    in 30s     

2023-05-25 18:22:52 (24.9 MB/s) - ‘/content/gdrive/MyDrive/controlnet_dataset/images.tar’ saved [793579520/793579520]



might take some time to unzip the whole image

In [ ]:
 !tar -C /content/gdrive/MyDrive/controlnet_dataset -xvf /content/gdrive/MyDrive/controlnet_dataset/images.tar

Streaming output truncated to the last 5000 lines.
Images/n02109047-Great_Dane/n02109047_34148.jpg
Images/n02109047-Great_Dane/n02109047_34162.jpg
Images/n02109047-Great_Dane/n02109047_34209.jpg
Images/n02109047-Great_Dane/n02109047_3643.jpg
Images/n02109047-Great_Dane/n02109047_4267.jpg
Images/n02109047-Great_Dane/n02109047_481.jpg
Images/n02109047-Great_Dane/n02109047_5618.jpg
Images/n02109047-Great_Dane/n02109047_5675.jpg
Images/n02109047-Great_Dane/n02109047_5822.jpg
Images/n02109047-Great_Dane/n02109047_5894.jpg
Images/n02109047-Great_Dane/n02109047_5936.jpg
Images/n02109047-Great_Dane/n02109047_6008.jpg
Images/n02109047-Great_Dane/n02109047_6265.jpg
Images/n02109047-Great_Dane/n02109047_6276.jpg
Images/n02109047-Great_Dane/n02109047_6447.jpg
Images/n02109047-Great_Dane/n02109047_6680.jpg
Images/n02109047-Great_Dane/n02109047_7072.jpg
Images/n02109047-Great_Dane/n02109047_7156.jpg
Images/n02109047-Great_Dane/n02109047_7227.jpg
Images/n02109047-Great_Dane/n02109047_744.jpg
Images/n

In [4]:
image_path = "/content/gdrive/MyDrive/controlnet_dataset/Images"
source_path = "/content"

In [5]:
os.chdir(image_path)
!ls

n02085620_10074.jpg  n02085620_1620.jpg  n02085620_3407.jpg
n02085620_10131.jpg  n02085620_1765.jpg  n02085620_3409.jpg
n02085620_10621.jpg  n02085620_1816.jpg  n02085620_3423.jpg
n02085620_1073.jpg   n02085620_1862.jpg  n02085620_3485.jpg
n02085620_10976.jpg  n02085620_1916.jpg  n02085620_3488.jpg
n02085620_11140.jpg  n02085620_199.jpg	 n02085620_3593.jpg
n02085620_11238.jpg  n02085620_2053.jpg  n02085620_3651.jpg
n02085620_11258.jpg  n02085620_2188.jpg  n02085620_3677.jpg
n02085620_11337.jpg  n02085620_2204.jpg  n02085620_3681.jpg
n02085620_11477.jpg  n02085620_2208.jpg  n02085620_368.jpg
n02085620_1152.jpg   n02085620_242.jpg	 n02085620_3742.jpg
n02085620_11696.jpg  n02085620_2479.jpg  n02085620_3763.jpg
n02085620_11818.jpg  n02085620_2507.jpg  n02085620_3826.jpg
n02085620_11948.jpg  n02085620_2517.jpg  n02085620_382.jpg
n02085620_1205.jpg   n02085620_2590.jpg  n02085620_3838.jpg
n02085620_12101.jpg  n02085620_2614.jpg  n02085620_3875.jpg
n02085620_12334.jpg  n02085620_2650.jpg  n02

Takes some time to move all images from sub folder to main folder

In [ ]:
os.system("find . -type f -print0 | xargs -0 mv -t .")
os.system("find . -type d -empty -delete")

0

NOTE: Please don't run below cell if you want to train on whole 20k images. Below cell only retain 100 images and deletes rest of them. This is just for the demo. Original model was trained on all images loacally. 

In [ ]:
command = "ls -1 " + image_path +" | tail -n +101 | xargs -I {} rm " + image_path + "/{}"
print(command)
os.system(command)

ls -1 /content/gdrive/MyDrive/controlnet_dataset/Images | tail -n +101 | xargs -I {} rm /content/gdrive/MyDrive/controlnet_dataset/Images/{}


0

In [6]:
!find . -type f | wc -l

100


In [3]:
os.chdir("/content/gdrive/MyDrive/controlnet_dataset/part-1")
!ls

annotator  captioner  __init__.py  requirements.txt


# Generate canny edge images and prompt using BLIP

In [4]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8.10
!sudo update-alternatives --config python3

update-alternatives: --install needs <link> <name> <path> <priority>

Use 'update-alternatives --help' for program usage information.
There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.10   2         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.8    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in manual mode


In [21]:
!sudo apt install python3-pip
!pip install -r requirements.txt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (20.0.2-5ubuntu1.8).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
! python --version
!python3 -m pip install timm==0.4.12

Python 3.8.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached timm-0.4.12-py3-none-any.whl (376 kB)
  Attempting uninstall: timm
    Found existing installation: timm 0.9.2
    Uninstalling timm-0.9.2:
      Successfully uninstalled timm-0.9.2


In [24]:
!python -c "from timm import list_models; print(list_models(pretrained=True)[:5])"

['adv_inception_v3', 'cait_m36_384', 'cait_m48_448', 'cait_s24_224', 'cait_s24_384']


In [25]:
!python run_generation.py

Downloading: 100% 226k/226k [00:00<00:00, 3.25MB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 115kB/s]
Downloading: 100% 455k/455k [00:00<00:00, 3.58MB/s]
Downloading: 100% 570/570 [00:00<00:00, 2.33MB/s]
100% 1.97G/1.97G [00:13<00:00, 152MB/s]
reshape position embedding from 196 to 576
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_11258.jpg
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_10976.jpg
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_1235.jpg
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_10074.jpg
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_1271.jpg
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_10621.jpg
processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02

In [23]:
import os
import sys
import json
from pathlib import Path
import shutil
import cv2

from annotator.util import resize_image, HWC3
from annotator.canny import CannyDetector

#custom class to encapsulate BLIP inference for caption generation
from captioner.blip_caption import BlipCaption


def get_images(path, image_path='images'):

    path = os.path.abspath(path)
    image_path = os.path.join(path, image_path)
    images = os.listdir(image_path)
    images = [os.path.join(image_path, image) for image in images]
    return images


def generate_dir(path, data_folder='dataset'):
    path = os.path.abspath(path)
    data_path = os.path.join(path, data_folder)

    source = (os.path.join(data_path, 'source'))
    target = (os.path.join(data_path, 'target'))
    prompt = (os.path.join(data_path, 'prompt.json'))

    Path(source).mkdir(parents=True, exist_ok=True)
    Path(target).mkdir(parents=True, exist_ok=True)

    return source, target, prompt


def get_canny(image, source_image_path):
    res = 512
    l = 100
    h = 200

    try:
        image = cv2.imread(image)
        image = resize_image(HWC3(image), res)
        model_canny = CannyDetector()
        result = model_canny(image, l, h)

    except:
        raise Exception(f"cannot convert {image} into edge through canny detector: check if file exists")

    cv2.imwrite(source_image_path, result)


def prompt_json(caption, source, target, json_file):
    json_dict = {'source':str(source), 'target':str(target), 'prompt':str(caption)}

    with open(json_file, 'a') as f:
        json.dump(json_dict, f)
        f.write('\n')
    f.close()

ModuleNotFoundError: ignored

In [11]:
path = '/content/gdrive/MyDrive/controlnet_dataset'
list_images = get_images(path, image_path='Images')
source, target, json_file = generate_dir(path)

#generate instance, load model
caption_generator = BlipCaption()

100%|██████████| 1.97G/1.97G [00:21<00:00, 100MB/s]


reshape position embedding from 196 to 576
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth


In [12]:
for image in list_images:
    print(f"processing image {image}")
    #set source image target image and copy
    target_image = os.path.join(target, os.path.basename(image))
    source_image = os.path.join(source, os.path.basename(image))
    shutil.copyfile(image, target_image)

    #generate canny images
    get_canny(target_image, source_image_path=source_image)

    #generate caption from target image
    caption = caption_generator.generate_caption(target_image)

    #write to prompt.json
    prompt_json(caption, source_image, target_image, json_file)



processing image /content/gdrive/MyDrive/controlnet_dataset/Images/n02085620_11258.jpg


RuntimeError: ignored